# 01_data_preparation.ipynb

## Notebook Purpose
This notebook is designed to load, clean, and preprocess historical cryptocurrency data. It will also calculate technical indicators that will be used for further analysis and model training.

## Instructions
1. **Import Necessary Libraries**:
   - Import `pandas` for data manipulation.
   - Import functions from `utils.py` for loading, preprocessing data, and calculating technical indicators.

2. **Load Data**:
   - Use the `load_data` function to load the CSV file containing historical cryptocurrency data.

3. **Preprocess Data**:
   - Use the `preprocess_data` function to clean and preprocess the loaded data.
   - Ensure any missing values are handled appropriately.

4. **Calculate Technical Indicators**:
   - Use the `calculate_indicators` function to add technical indicators (e.g., SMA, EMA, RSI) to the data.

5. **Save Preprocessed Data**:
   - Save the cleaned and preprocessed data, including the calculated technical indicators, to a new CSV file for later use.

6. **Review Data**:
   - Display the first few rows of the preprocessed data to ensure it looks correct.

## Example Code
```python
# Import necessary libraries
import pandas as pd
from scripts.utils import load_data, preprocess_data, calculate_indicators

# Load data
data_path = 'data/historical_data/btc_usd.csv'  # Update this path based on the selected cryptocurrency
data = load_data(data_path)

# Preprocess data
data = preprocess_data(data)

# Calculate technical indicators
data = calculate_indicators(data)

# Save the preprocessed data
data.to_csv('data/historical_data/btc_usd_preprocessed.csv')

# Display the first few rows of the preprocessed data
data.head()


In [9]:
# Cell 1: Import necessary libraries and verify
try:
    import pandas as pd
    import numpy as np
    from pandas_datareader import data as pdr
    from datetime import datetime
    import matplotlib.pyplot as plt
    import seaborn as sns
    import requests
    from dotenv import load_dotenv
    import os
    import ccxt
    %matplotlib inline
    print("Libraries loaded successfully. Let's proceed!")
except ImportError as e:
    print(f"Uh-oh! Please verify the installation of: {e.name}")

Libraries loaded successfully. Let's proceed!


In [10]:
# Cell 2: Load environment variables and fetch API keys
import os
from dotenv import load_dotenv
import requests
import ccxt

# Load environment variables
load_dotenv()

# Fetch API keys
YAHOO_FINANCE_API_KEY = os.getenv("YAHOO_FINANCE_API_KEY")
COINBASE_API_KEY = os.getenv("COINBASE_API_KEY")
COINBASE_API_SECRET = os.getenv("COINBASE_API_SECRET")
ALPACA_API_KEY = os.getenv("ALPACA_API_KEY_ID")
ALPACA_API_SECRET = os.getenv("ALPACA_SECRET_KEY")
ALPACA_BASE_URL = os.getenv("ALPACA_BASE_URL")
ALPHA_VANTAGE_API_KEY = os.getenv("ALPHA_VANTAGE_API_KEY")
COINMARKETCAP_API_KEY = os.getenv("COINMARKETCAP_API_KEY")
CRYPTOCOMPARE_API_KEY = os.getenv("CRYPTOCOMPARE_API_KEY")

# Function to test API connection for each service
def test_api_connection():
    report = []

    # Test Yahoo Finance API connection
    print("Testing Yahoo Finance connection...")
    if YAHOO_FINANCE_API_KEY:
        try:
            response = requests.get("https://yfapi.net/v8/finance/chart/BTC-USD", headers={"x-api-key": YAHOO_FINANCE_API_KEY})
            if response.status_code == 200:
                report.append("Yahoo Finance: Connection successful.")
            else:
                report.append(f"Yahoo Finance: Failed to connect. Status code: {response.status_code}, Message: {response.text}")
        except Exception as e:
            report.append(f"Yahoo Finance: Failed to connect. Error: {str(e)}")
    else:
        report.append("Yahoo Finance: API key missing.")
    
    # Test Coinbase API connection using ccxt
    try:
        print("Testing Coinbase connection...")
        coinbase = ccxt.coinbase({
            'apiKey': COINBASE_API_KEY,
            'secret': COINBASE_API_SECRET
        })
        print("Coinbase object created.")
        markets = coinbase.load_markets()
        print("Coinbase markets loaded.")
        balance = coinbase.fetch_balance()
        print("Coinbase balance fetched.")
        report.append("Coinbase: Connection successful.")
    except Exception as e:
        print("Error during Coinbase connection test:")
        print(e)
        report.append(f"Coinbase: Failed to connect. Error: {str(e)}")
    
    # Test Alpaca API connection using ccxt
    try:
        print("Testing Alpaca connection...")
        alpaca = ccxt.alpaca({
            'apiKey': ALPACA_API_KEY,
            'secret': ALPACA_API_SECRET,
            'enableRateLimit': True,
            'options': {
                'defaultType': 'account',
                'baseUrl': ALPACA_BASE_URL
            }
        })
        print("Alpaca object created.")
        time = alpaca.fetch_time()
        print("Alpaca time fetched.")
        report.append("Alpaca: Connection successful.")
    except Exception as e:
        print("Error during Alpaca connection test:")
        print(e)
        report.append(f"Alpaca: Failed to connect. Error: {str(e)}")
    
    # Test Alpha Vantage API connection
    print("Testing Alpha Vantage connection...")
    if ALPHA_VANTAGE_API_KEY:
        try:
            response = requests.get(f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=IBM&apikey={ALPHA_VANTAGE_API_KEY}")
            if response.status_code == 200:
                report.append("Alpha Vantage: Connection successful.")
            else:
                report.append(f"Alpha Vantage: Failed to connect. Status code: {response.status_code}, Message: {response.text}")
        except Exception as e:
            report.append(f"Alpha Vantage: Failed to connect. Error: {str(e)}")
    else:
        report.append("Alpha Vantage: API key missing.")
    
    # Test CoinMarketCap API connection
    print("Testing CoinMarketCap connection...")
    if COINMARKETCAP_API_KEY:
        try:
            response = requests.get(f"https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest", headers={"X-CMC_PRO_API_KEY": COINMARKETCAP_API_KEY})
            if response.status_code == 200:
                report.append("CoinMarketCap: Connection successful.")
            else:
                report.append(f"CoinMarketCap: Failed to connect. Status code: {response.status_code}, Message: {response.text}")
        except Exception as e:
            report.append(f"CoinMarketCap: Failed to connect. Error: {str(e)}")
    else:
        report.append("CoinMarketCap: API key missing.")
    
    # Test CryptoCompare API connection
    print("Testing CryptoCompare connection...")
    if CRYPTOCOMPARE_API_KEY:
        try:
            response = requests.get(f"https://min-api.cryptocompare.com/data/pricemulti?fsyms=BTC&tsyms=USD", headers={"authorization": f"Apikey {CRYPTOCOMPARE_API_KEY}"})
            if response.status_code == 200:
                report.append("CryptoCompare: Connection successful.")
            else:
                report.append(f"CryptoCompare: Failed to connect. Status code: {response.status_code}, Message: {response.text}")
        except Exception as e:
            report.append(f"CryptoCompare: Failed to connect. Error: {str(e)}")
    else:
        report.append("CryptoCompare: API key missing.")
    
    # Print connection report
    for line in report:
        print(line)

# Run the API connection test
test_api_connection()

Testing Yahoo Finance connection...
Testing Coinbase connection...
Coinbase object created.
Error during Coinbase connection test:
index out of range
Testing Alpaca connection...
Alpaca object created.
Error during Alpaca connection test:
alpaca {"code":40110000,"message":"request is not authorized"}
Testing Alpha Vantage connection...
Testing CoinMarketCap connection...
Testing CryptoCompare connection...
Yahoo Finance: Failed to connect. Status code: 403, Message: {"message":"Forbidden","hint":"Sign up for API key https://financeapi.net/tutorial"}
Coinbase: Failed to connect. Error: index out of range
Alpaca: Failed to connect. Error: alpaca {"code":40110000,"message":"request is not authorized"}
Alpha Vantage: Connection successful.
CoinMarketCap: Connection successful.
CryptoCompare: Connection successful.


In [12]:
# Cell 3: Function to fetch historical data from Yahoo Finance
def fetch_yahoo_finance_data(ticker, start_date, end_date):
    base_url = "https://yfapi.net/v8/finance/chart/"
    url = f"{base_url}{ticker}-USD"
    
    headers = {
        "x-api-key": YAHOO_FINANCE_API_KEY
    }
    
    params = {
        "symbol": f"{ticker}-USD",
        "period1": int(datetime.strptime(start_date, "%Y-%m-%d").timestamp()),
        "period2": int(datetime.strptime(end_date, "%Y-%m-%d").timestamp()),
        "interval": "1d"
    }
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        data = response.json()
        if 'chart' in data and 'result' in data['chart'] and data['chart']['result']:
            result = data['chart']['result'][0]
            timestamps = result['timestamp']
            indicators = result['indicators']['quote'][0]
            df = pd.DataFrame({
                'time': pd.to_datetime(timestamps, unit='s'),
                'open': indicators['open'],
                'high': indicators['high'],
                'low': indicators['low'],
                'close': indicators['close'],
                'volume': indicators['volume']
            })
            return df
        else:
            print(f"No data available for {ticker} from Yahoo Finance")
            return None
    else:
        print(f"Failed to fetch data from Yahoo Finance: {response.status_code}")
        return None

In [13]:
# Cell 4: Function to fetch historical data from Coinbase using ccxt
def fetch_coinbase_data(currency_pair, start_date, end_date):
    try:
        print(f"Creating Coinbase object for {currency_pair}...")
        coinbase = ccxt.coinbase({
            'apiKey': COINBASE_API_KEY,
            'secret': COINBASE_API_SECRET
        })
        print("Coinbase object created.")
        
        markets = coinbase.load_markets()
        print(f"Markets loaded: {len(markets)} markets available.")
        print("Markets detail:", list(markets.items())[:5])  # Print a few market details for inspection

        if len(markets) == 0:
            raise ValueError("No markets loaded.")

        since = coinbase.parse8601(f'{start_date}T00:00:00Z')
        end = coinbase.parse8601(f'{end_date}T00:00:00Z')
        print(f"Fetching OHLCV data from {start_date} to {end_date} for {currency_pair}...")

        ohlcv = coinbase.fetch_ohlcv(currency_pair + '/USD', '1d', since)

        if not ohlcv:
            print(f"No data returned for {currency_pair} from {start_date} to {end_date}")
            return None

        print("Data fetched successfully, creating DataFrame...")
        df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df['time'] = pd.to_datetime(df['timestamp'], unit='ms')
        df = df[['time', 'open', 'high', 'low', 'close', 'volume']]
        
        return df
    except Exception as e:
        print(f"Failed to fetch data from Coinbase: {str(e)}")
        return None

In [14]:
# Cell 5: Function to fetch historical data from Alpaca
def fetch_alpaca_crypto_data(symbol, start_date, end_date):
    url = f"{ALPACA_BASE_URL}/v1beta2/crypto/bars"
    
    headers = {
        "APCA-API-KEY-ID": ALPACA_API_KEY,
        "APCA-API-SECRET-KEY": ALPACA_API_SECRET
    }
    
    params = {
        'start': start_date,
        'end': end_date,
        'timeframe': '1Day',
        'symbols': symbol
    }
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        data = response.json()
        if 'bars' in data:
            bars = data['bars'][symbol]
            df = pd.DataFrame(bars)
            df['time'] = pd.to_datetime(df['t'])
            df = df.rename(columns={'o': 'open', 'h': 'high', 'l': 'low', 'c': 'close', 'v': 'volume'})
            df = df[['time', 'open', 'high', 'low', 'close', 'volume']]
            return df
        else:
            print(f"No data available for {symbol} from Alpaca")
            return None
    else:
        print(f"Failed to fetch data from Alpaca: {response.status_code}")
        return None

In [21]:
# Cell 6: Function to fetch historical data from Alpha Vantage
def fetch_alpha_vantage_data(symbol):
    base_url = "https://www.alphavantage.co/query"
    params = {
        "function": "DIGITAL_CURRENCY_DAILY",
        "symbol": symbol,
        "market": "USD",
        "apikey": ALPHA_VANTAGE_API_KEY
    }
    
    response = requests.get(base_url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        if 'Time Series (Digital Currency Daily)' in data:
            time_series = data['Time Series (Digital Currency Daily)']
            df = pd.DataFrame.from_dict(time_series, orient='index')
            df = df.rename(columns={
                '1a. open (USD)': 'open',
                '2a. high (USD)': 'high',
                '3a. low (USD)': 'low',
                '4a. close (USD)': 'close',
                '5. volume': 'volume'
            })
            df.index = pd.to_datetime(df.index)
            df.reset_index(inplace=True)
            df = df.rename(columns={'index': 'time'})
            
            # Debug print to check available columns
            print(f"Alpha Vantage Data Columns for {symbol}: {df.columns}")
            
            # Check if the required columns are in the DataFrame
            if all(col in df.columns for col in ['time', 'open', 'high', 'low', 'close', 'volume']):
                df = df[['time', 'open', 'high', 'low', 'close', 'volume']]
                df = df.astype({
                    'open': 'float',
                    'high': 'float',
                    'low': 'float',
                    'close': 'float',
                    'volume': 'float'
                })
                return df
            else:
                print(f"Missing expected columns in Alpha Vantage data for {symbol}")
                return None
        else:
            print(f"No 'Time Series (Digital Currency Daily)' data found for {symbol}")
            return None
    else:
        print(f"Failed to fetch data from Alpha Vantage: {response.status_code}")
        return None


In [22]:
# Cell 7: Function to fetch historical data from CoinMarketCap
def fetch_coinmarketcap_data(symbol, start_date, end_date):
    base_url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/ohlcv/historical"
    params = {
        "symbol": symbol,
        "convert": "USD",
        "time_start": int(datetime.strptime(start_date, "%Y-%m-%d").timestamp()),
        "time_end": int(datetime.strptime(end_date, "%Y-%m-%d").timestamp())
    }
    
    headers = {
        "X-CMC_PRO_API_KEY": COINMARKETCAP_API_KEY
    }
    
    response = requests.get(base_url, headers=headers, params=params)
    
    if response.status_code == 200:
        data = response.json()
        if 'data' in data and 'quotes' in data['data']:
            quotes = data['data']['quotes']
            df = pd.DataFrame(quotes)
            df['time'] = pd.to_datetime(df['timestamp'], unit='s')
            df = df.rename(columns={'open': 'open', 'high': 'high', 'low': 'low', 'close': 'close', 'volume': 'volume'})
            df = df[['time', 'open', 'high', 'low', 'close', 'volume']]
            return df
        else:
            print(f"No data available for {symbol} from CoinMarketCap")
            return None
    else:
        print(f"Failed to fetch data from CoinMarketCap: {response.status_code}")
        return None

In [23]:
# Cell 8: Function to fetch historical data from CryptoCompare
def fetch_cryptocompare_data(symbol, start_date, end_date):
    base_url = f"https://min-api.cryptocompare.com/data/v2/histoday"
    params = {
        "fsym": symbol,
        "tsym": "USD",
        "toTs": int(datetime.strptime(end_date, "%Y-%m-%d").timestamp()),
        "limit": 2000,  # CryptoCompare allows fetching up to 2000 days in one call
        "api_key": CRYPTOCOMPARE_API_KEY
    }
    
    response = requests.get(base_url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        if 'Data' in data and 'Data' in data['Data']:
            data = data['Data']['Data']
            df = pd.DataFrame(data)
            df['time'] = pd.to_datetime(df['time'], unit='s')
            df = df.rename(columns={'open': 'open', 'high': 'high', 'low': 'low', 'close': 'close', 'volumeto': 'volume'})
            df = df[['time', 'open', 'high', 'low', 'close', 'volume']]
            return df
        else:
            print(f"No data available for {symbol} from CryptoCompare")
            return None
    else:
        print(f"Failed to fetch data from CryptoCompare: {response.status_code}")
        return None

In [24]:
# Cell 9: Function to save data to a CSV file
def save_data_to_csv(data, filename):
    if data is not None and not data.empty:
        data.to_csv(filename, index=False)
        print(f"Data saved to {filename}")
    else:
        print(f"No data to save for {filename}")

In [25]:
# Cell 10: Fetch and save data for cryptocurrencies year by year
cryptos = ["BTC", "ETH", "SOL"]
years = range(2018, 2024)

# Create the historical_data folder if it doesn't exist
os.makedirs("../data/historical_data", exist_ok=True)

for crypto in cryptos:
    for year in years:
        start_date = f"{year}-01-01"
        end_date = f"{year}-12-31"
        
        print(f"Fetching data for {crypto} for the year {year}...")

        # Fetch data from Yahoo Finance
        yahoo_data = fetch_yahoo_finance_data(crypto, start_date, end_date)
        if yahoo_data is not None:
            save_data_to_csv(yahoo_data, f"../data/historical_data/{crypto}_yahoo_{year}.csv")

        # Fetch data from Coinbase
        coinbase_data = fetch_coinbase_data(crypto, start_date, end_date)
        if coinbase_data is not None:
            save_data_to_csv(coinbase_data, f"../data/historical_data/{crypto}_coinbase_{year}.csv")

        # Fetch data from Alpaca
        alpaca_data = fetch_alpaca_crypto_data(crypto, start_date, end_date)
        if alpaca_data is not None:
            save_data_to_csv(alpaca_data, f"../data/historical_data/{crypto}_alpaca_{year}.csv")

        # Fetch data from Alpha Vantage
        alpha_vantage_data = fetch_alpha_vantage_data(crypto)
        if alpha_vantage_data is not None:
            save_data_to_csv(alpha_vantage_data, f"../data/historical_data/{crypto}_alpha_vantage_{year}.csv")

        # Fetch data from CoinMarketCap
        coinmarketcap_data = fetch_coinmarketcap_data(crypto, start_date, end_date)
        if coinmarketcap_data is not None:
            save_data_to_csv(coinmarketcap_data, f"../data/historical_data/{crypto}_coinmarketcap_{year}.csv")

        # Fetch data from CryptoCompare
        cryptocompare_data = fetch_cryptocompare_data(crypto, start_date, end_date)
        if cryptocompare_data is not None:
            save_data_to_csv(cryptocompare_data, f"../data/historical_data/{crypto}_cryptocompare_{year}.csv")


Fetching data for BTC for the year 2018...
Failed to fetch data from Yahoo Finance: 403
Creating Coinbase object for BTC...
Coinbase object created.
Failed to fetch data from Coinbase: index out of range
Failed to fetch data from Alpaca: 404
Alpha Vantage Data Columns for BTC: Index(['time', '1. open', '2. high', '3. low', '4. close', 'volume'], dtype='object')
Missing expected columns in Alpha Vantage data for BTC
Failed to fetch data from CoinMarketCap: 403
Data saved to ../data/historical_data/BTC_cryptocompare_2018.csv
Fetching data for BTC for the year 2019...
Failed to fetch data from Yahoo Finance: 403
Creating Coinbase object for BTC...
Coinbase object created.
Failed to fetch data from Coinbase: index out of range
Failed to fetch data from Alpaca: 404
Alpha Vantage Data Columns for BTC: Index(['time', '1. open', '2. high', '3. low', '4. close', 'volume'], dtype='object')
Missing expected columns in Alpha Vantage data for BTC
Failed to fetch data from CoinMarketCap: 403
Data sa

In [26]:
# Cell 11: Verify the data saving process
for crypto in cryptos:
    files_saved = [
        f"../data/historical_data/{crypto}_yahoo_{year}.csv",
        f"../data/historical_data/{crypto}_coinbase_{year}.csv",
        f"../data/historical_data/{crypto}_alpaca_{year}.csv",
        f"../data/historical_data/{crypto}_alpha_vantage_{year}.csv",
        f"../data/historical_data/{crypto}_coinmarketcap_{year}.csv",
        f"../data/historical_data/{crypto}_cryptocompare_{year}.csv"
    ]
    
    for file in files_saved:
        if os.path.exists(file):
            print(f"Verification successful: {file} exists.")
        else:
            print(f"Verification failed: {file} does not exist.")

Verification failed: ../data/historical_data/BTC_yahoo_2023.csv does not exist.
Verification failed: ../data/historical_data/BTC_coinbase_2023.csv does not exist.
Verification failed: ../data/historical_data/BTC_alpaca_2023.csv does not exist.
Verification failed: ../data/historical_data/BTC_alpha_vantage_2023.csv does not exist.
Verification failed: ../data/historical_data/BTC_coinmarketcap_2023.csv does not exist.
Verification successful: ../data/historical_data/BTC_cryptocompare_2023.csv exists.
Verification failed: ../data/historical_data/ETH_yahoo_2023.csv does not exist.
Verification failed: ../data/historical_data/ETH_coinbase_2023.csv does not exist.
Verification failed: ../data/historical_data/ETH_alpaca_2023.csv does not exist.
Verification failed: ../data/historical_data/ETH_alpha_vantage_2023.csv does not exist.
Verification failed: ../data/historical_data/ETH_coinmarketcap_2023.csv does not exist.
Verification successful: ../data/historical_data/ETH_cryptocompare_2023.csv 